In [1]:
import os
import numpy as np
import pandas as pd
from utils.utils import *
from utils.kabsch2D import *
from nsd_access import NSDAccess 
from scipy import stats

In [2]:
nsda = NSDAccess(nsd_dir)

In [3]:
def create_rotation_df(subj, rois, sess):
    cols = ['source', 'base', 'target', 'U', 'distance']
    rotations = np.zeros(((len(rois) * (len(rois)-1)) * 2, len(cols)), dtype=object)
    i = 0 
    for roi_source in rois.keys():
        mds_source = np.load(f'/home/stan/thesis-repo/data/MDS/{subj}/{subj}_{sess}_{roi_source}_mds_betas_train.npy', allow_pickle=False)
        for roi_target in rois.keys():
            if roi_source != roi_target:
                for j in range(2):
                    mds_target = np.load(f'/home/stan/thesis-repo/data/MDS/{subj}/{subj}_{sess}_{roi_target}_mds_betas_train.npy', allow_pickle=False)
                    # flip the target when j is at 1, does nothing when j = 0 
                    rotations[i, 1] = roi_target
                    if j == 1:
                        mds_target = np.dot(mds_target, np.array([[-1, 0], [0, 1]]))
                        roi_target = roi_target + "_flipped"
                    U, t= kabsch2D(mds_source, mds_target, translate=True )
                    rotations[i, 0] = roi_source
                    rotations[i, 2] = roi_target
                    rotations[i, 3] = U
                    rotated_source = rotate(mds_source, U)
                    rotations[i, 4] = avg_distance(rotated_source, mds_target, t)
                    i += 1
    df = pd.DataFrame(rotations, columns=cols)
    df = df.astype({'distance': 'float32'}) # needed for later indexing
    return df

# taking distance instead of error + shuffling images and checking for chance 

In [4]:
def get_ranking(df):
    # get the index of the lowest error for each pair in each ROI
    indx_to_keep = df.groupby(['source', 'base'])['distance'].idxmin()
    df_filtered = df.loc[indx_to_keep]
    rank = df_filtered.groupby('source')['distance'].mean().sort_values()
    return rank

https://www.sciencedirect.com/science/article/pii/S0167715298000066
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.weightedtau.html

Its cool but doesn't give a p value

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kendalltau.html#scipy.stats.kendalltau 
Normal Kendall Tau might be the thing?? 

I asked chatGPT, and it tells me to use Friedman test, by looking at the difference between the average rank

In [5]:
rois

{'V1': 1,
 'V2': 2,
 'V3': 3,
 'hV4': 4,
 'VO-1': 5,
 'VO-2': 6,
 'PHC-1': 7,
 'PHC-2': 8,
 'LO-1': 9,
 'LO-2': 10,
 'TO-1': 11,
 'TO-2': 12}

In [6]:
def get_rank_dict(subj_list, rois, sessions):
    dict, dict_values, dict_matrices = {}, {}, {}
    for i in range(len(subj_list)):
        rotations_df = create_rotation_df(subj=subj_list[i], rois=rois, sess=sessions[i])
      #  print(rotations_df)
        rank = get_ranking(rotations_df)
        rank_int = [rois[roi] for roi in rank.index]
        dict[subj_list[i]] = rank_int
        dict_values[subj_list[i]] = rank

    return dict, dict_values

In [7]:
subj_list = [f'subj0{i}' for i in range(1,9)]
sessions = [37, 37, 29, 27, 37, 29, 37, 27]

rank_dict, rank_dict_values = get_rank_dict(subj_list=subj_list, rois=rois, sessions=sessions)
rank_dict

{'subj01': [10, 5, 7, 6, 8, 9, 3, 11, 2, 12, 4, 1],
 'subj02': [8, 6, 7, 5, 11, 10, 9, 3, 2, 12, 4, 1],
 'subj03': [6, 3, 7, 4, 11, 8, 2, 5, 9, 1, 10, 12],
 'subj04': [5, 6, 7, 8, 11, 9, 4, 10, 3, 2, 12, 1],
 'subj05': [7, 5, 8, 6, 9, 10, 3, 4, 11, 12, 2, 1],
 'subj06': [7, 8, 3, 10, 9, 6, 5, 2, 11, 12, 4, 1],
 'subj07': [3, 2, 1, 6, 4, 5, 7, 9, 11, 10, 12, 8],
 'subj08': [3, 2, 7, 6, 4, 9, 5, 10, 1, 8, 11, 12]}

In [8]:
rank_dict_values.values()

dict_values([source
LO-2     0.641302
VO-1     0.669528
PHC-1    0.679085
VO-2     0.683722
PHC-2    0.687655
LO-1     0.688435
V3       0.710606
TO-1     0.712962
V2       0.715576
TO-2     0.717176
hV4      0.731694
V1       0.751698
Name: distance, dtype: float32, source
PHC-2    0.614185
VO-2     0.621779
PHC-1    0.626831
VO-1     0.634687
TO-1     0.641044
LO-2     0.653578
LO-1     0.677640
V3       0.687868
V2       0.693080
TO-2     0.711625
hV4      0.735202
V1       0.736357
Name: distance, dtype: float32, source
VO-2     0.598384
V3       0.621024
PHC-1    0.629678
hV4      0.638733
TO-1     0.651264
PHC-2    0.651845
V2       0.653212
VO-1     0.667324
LO-1     0.676175
V1       0.683548
LO-2     0.708849
TO-2     0.729552
Name: distance, dtype: float32, source
VO-1     0.594328
VO-2     0.612262
PHC-1    0.627951
PHC-2    0.630846
TO-1     0.648271
LO-1     0.669451
hV4      0.671770
LO-2     0.672740
V3       0.688193
V2       0.702060
TO-2     0.704637
V1       0.716105

In [9]:
corr = np.zeros((len(subj_list), len(subj_list)))
p_values = np.zeros((len(subj_list), len(subj_list)))

for i in range(len(subj_list)):
    for j in range(len(subj_list)):
        res = stats.kendalltau(list(rank_dict.values())[i], list(rank_dict.values())[j])
        corr[i, j] = res.correlation
        p_values[i, j] = round(res.pvalue, 5)
        

In [10]:
corr_df = pd.DataFrame(corr, columns=rank_dict.keys(), index=rank_dict.keys())
corr_df

,subj01,subj02,subj03,subj04,subj05,subj06,subj07,subj08
subj01,1.000000,0.484848,-0.272727,0.242424,0.424242,0.121212,-0.121212,0.090909
subj02,0.484848,1.000000,-0.303030,0.030303,0.454545,0.333333,-0.212121,-0.121212
subj03,-0.272727,-0.303030,1.000000,0.242424,0.000000,-0.242424,0.060606,0.212121
subj04,0.242424,0.030303,0.242424,1.000000,-0.090909,-0.151515,-0.090909,0.181818
subj05,0.424242,0.454545,0.000000,-0.090909,1.000000,0.575758,-0.090909,-0.303030
subj06,0.121212,0.333333,-0.242424,-0.151515,0.575758,1.000000,0.030303,-0.484848
subj07,-0.121212,-0.212121,0.060606,-0.090909,-0.090909,0.030303,1.000000,0.303030
subj08,0.090909,-0.121212,0.212121,0.181818,-0.303030,-0.484848,0.303030,1.000000


In [11]:
pvalues_df = pd.DataFrame(p_values, columns=rank_dict.keys(), index=rank_dict.keys())
pvalues_df

,subj01,subj02,subj03,subj04,subj05,subj06,subj07,subj08
subj01,0.00000,0.03105,0.24958,0.31081,0.06287,0.63836,0.63836,0.73731
subj02,0.03105,0.00000,0.19695,0.94656,0.04474,0.15259,0.38070,0.63836
subj03,0.24958,0.19695,0.00000,0.31081,1.00000,0.31081,0.84059,0.38070
subj04,0.31081,0.94656,0.31081,0.00000,0.73731,0.54520,0.73731,0.45902
subj05,0.06287,0.04474,1.00000,0.73731,0.00000,0.00876,0.73731,0.19695
subj06,0.63836,0.15259,0.31081,0.54520,0.00876,0.00000,0.94656,0.03105
subj07,0.63836,0.38070,0.84059,0.73731,0.73731,0.94656,0.00000,0.19695
subj08,0.73731,0.63836,0.38070,0.45902,0.19695,0.03105,0.19695,0.00000


In [12]:
rank_total = np.zeros(len(rois))
for i in range(len(subj_list)):
    rank_total += list(rank_dict_values.values())[i]

In [13]:
rank_dict_values

{'subj01': source
 LO-2     0.641302
 VO-1     0.669528
 PHC-1    0.679085
 VO-2     0.683722
 PHC-2    0.687655
 LO-1     0.688435
 V3       0.710606
 TO-1     0.712962
 V2       0.715576
 TO-2     0.717176
 hV4      0.731694
 V1       0.751698
 Name: distance, dtype: float32,
 'subj02': source
 PHC-2    0.614185
 VO-2     0.621779
 PHC-1    0.626831
 VO-1     0.634687
 TO-1     0.641044
 LO-2     0.653578
 LO-1     0.677640
 V3       0.687868
 V2       0.693080
 TO-2     0.711625
 hV4      0.735202
 V1       0.736357
 Name: distance, dtype: float32,
 'subj03': source
 VO-2     0.598384
 V3       0.621024
 PHC-1    0.629678
 hV4      0.638733
 TO-1     0.651264
 PHC-2    0.651845
 V2       0.653212
 VO-1     0.667324
 LO-1     0.676175
 V1       0.683548
 LO-2     0.708849
 TO-2     0.729552
 Name: distance, dtype: float32,
 'subj04': source
 VO-1     0.594328
 VO-2     0.612262
 PHC-1    0.627951
 PHC-2    0.630846
 TO-1     0.648271
 LO-1     0.669451
 hV4      0.671770
 LO-2     0.

In [14]:
rank_total.sort_values()

source
PHC-1    5.165553
VO-2     5.177632
VO-1     5.266647
V3       5.337876
PHC-2    5.358953
LO-1     5.431395
LO-2     5.438933
V2       5.484544
TO-1     5.486264
hV4      5.539582
V1       5.751069
TO-2     5.789586
Name: distance, dtype: float64

In [17]:
# apply rotation on one subject
def apply_rotation(target, subj, rois, sessions):
    # obtain the rotation matrix again
    rotations_df = create_rotation_df(subj, rois, sessions)
    rotations_df = rotations_df[rotations_df['base'] == target]
    indx_to_keep = rotations_df.groupby(['source', 'base'])['distance'].idxmin()
    rotations_df = rotations_df.loc[indx_to_keep]
    rotations_df = rotations_df.reset_index(drop=True)
    print(rotations_df.U)
    rotated_path = os.path.join(proj_dir, 'MDS_rotated', subj)
    print(rotated_path)
    if not os.path.exists(rotated_path):
        os.makedirs(rotated_path)
        print(os.path.exists(rotated_path))
    for roi in rois.keys():
        source_mds = np.load(f'/home/stan/thesis-repo/data/MDS/{subj}/{subj}_{sessions}_{roi}_mds_betas_train.npy', allow_pickle=False)
        mds_rotated_file = os.path.join(rotated_path, f'{subj}_{sessions}_{roi}_mds_betas_train_rotated_{target}.npy')
        # if the source is our reference point, just save it without applying rotation
        if roi == target:
            if not os.path.exists(mds_rotated_file):
                np.save(mds_rotated_file, source_mds, allow_pickle=True)
            continue 
        U = rotations_df.loc[rotations_df['source'] == roi, 'U'].squeeze()
        mds_rotated = np.dot(source_mds, U)  # apply rotation with dot product
        if not os.path.exists(mds_rotated_file):
            np.save(mds_rotated_file, mds_rotated, allow_pickle=True)
    
    

In [18]:
for i in range(1, 9):
    apply_rotation('PHC-1', f'subj0{i}', rois, sessions[i-1])


0     [[0.3854254, -0.9227389], [0.9227389, 0.3854254]]
1     [[-0.7893548309175643, -0.613937253232856], [0...
2     [[0.98475903, 0.17392436], [-0.17392436, 0.984...
3     [[0.86215484, -0.50664485], [0.50664485, 0.862...
4     [[-0.98772126, -0.15622663], [0.15622663, -0.9...
5     [[-0.6911771960423665, 0.7226853282522155], [-...
6     [[0.65729177, -0.75363624], [0.75363624, 0.657...
7     [[0.7329165, -0.6803186], [0.6803186, 0.7329165]]
8     [[0.13863223248103285, 0.9903439322363339], [-...
9     [[-0.6130681307201219, -0.7900300418941899], [...
10    [[-0.7989624207084763, -0.6013809527210279], [...
Name: U, dtype: object
/home/stan/thesis-repo/projects/NSD/MDS_rotated/subj01
0     [[-0.19842188, -0.9801167], [0.9801167, -0.198...
1     [[0.99857616, -0.053343683], [0.053343683, 0.9...
2     [[0.91326874, 0.40735754], [-0.40735754, 0.913...
3     [[0.9951258, 0.09861478], [-0.09861478, 0.9951...
4     [[-0.610746865004843, 0.7918259069307825], [-0...
5     [[-0.87343055, 0.486

In [19]:
# for all of rotated MDS: try to center them? Find the highest value from 0 